In [1]:
import json
from datetime import date

import polars as pl
from polars import selectors as cs


In [2]:
dataset1 = pl.read_csv("data/dataset_1.csv")
dataset2 = pl.read_csv("data/dataset_2.tsv", separator="\t")

neighborhoods = pl.read_csv("data/institute_neighborhoods.csv")
last_visit_info = pl.read_csv("data/last_visit_info.csv")

In [3]:
dataset1.head()

Patient Id,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Patient First Name,Family Name,Father's name,Mother's age,Father's age,Institute Name,Location of Institute,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Test 1,Test 2,Test 3,Test 4,Test 5,Parental consent,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
str,f64,str,str,str,str,f64,str,str,str,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,str,str
"""PID0x6418""",2.0,"""Yes""","""No""","""Yes""","""No""",4.760603,"""Richard""",null,"""Larre""",null,null,"""Boston Specialty & Rehabilitat…","""55 FRUIT ST CENTRAL, MA 02114 …","""Alive""","""Normal (30-60)""","""Normal""",0.0,null,null,1.0,0.0,"""Yes""","""High""",null,null,"""Not applicable""","""Institute""","""No""",null,"""No""","""No""","""No""","""Yes""",null,null,9.857562,null,1.0,1.0,1.0,1.0,1.0,"""Mitochondrial genetic inherita…","""Leber's hereditary optic neuro…"
"""PID0x25d5""",4.0,"""Yes""","""Yes""","""No""","""No""",4.910669,"""Mike""",null,"""Brycen""",null,23.0,"""St. Margaret's Hospital For Wo…","""1515 COMMONWEALTH AV ALLSTON/B…","""Deceased""","""Tachypnea""","""Normal""",null,0.0,0.0,1.0,0.0,"""Yes""","""High""",null,"""No""","""None""",null,"""Yes""","""Yes""","""Not applicable""","""Not applicable""","""No""","""Yes""",null,"""Multiple""",5.52256,"""normal""",1.0,null,1.0,1.0,0.0,null,"""Cystic fibrosis"""
"""PID0x4a82""",6.0,"""Yes""","""No""","""No""","""No""",4.893297,"""Kimberly""",null,"""Nashon""",41.0,22.0,null,"""-""","""Alive""","""Normal (30-60)""","""Tachycardia""",0.0,0.0,0.0,1.0,0.0,"""Yes""","""Low""",null,"""No record""","""Not applicable""",null,"""Yes""","""No""","""Yes""",null,"""Yes""","""Yes""",4.0,"""Singular""",null,"""normal""",0.0,1.0,1.0,1.0,1.0,"""Multifactorial genetic inherit…","""Diabetes"""
"""PID0x4ac8""",12.0,"""Yes""","""No""","""Yes""","""No""",4.70528,"""Jeffery""","""Hoelscher""","""Aayaan""",21.0,null,null,"""55 FRUIT ST CENTRAL, MA 02114 …","""Deceased""","""Tachypnea""","""Normal""",0.0,0.0,0.0,1.0,0.0,"""Yes""","""High""","""Male""","""Not available""","""No""","""Institute""","""No""","""Yes""","""-""","""Not applicable""",null,"""Yes""",1.0,"""Singular""",7.919321,"""inconclusive""",0.0,0.0,1.0,0.0,0.0,"""Mitochondrial genetic inherita…","""Leigh syndrome"""
"""PID0x1bf7""",11.0,"""Yes""","""No""",null,"""Yes""",4.720703,"""Johanna""","""Stutzman""","""Suave""",32.0,null,"""Carney Hospital""","""300 LONGWOOD AV FENWAY/KENMORE…","""Alive""","""Tachypnea""","""Tachycardia""",0.0,0.0,0.0,1.0,0.0,null,"""Low""","""Male""","""Not available""","""Not applicable""","""Institute""","""No""","""Yes""","""-""","""Not applicable""","""Yes""","""No""",4.0,"""Multiple""",4.09821,null,0.0,0.0,0.0,0.0,null,"""Multifactorial genetic inherit…","""Cancer"""


In [4]:
dataset2.head()

Patient Id,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Patient First Name,Family Name,Father's name,Mother's age,Father's age,Institute Name,Location of Institute,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Test 1,Test 2,Test 3,Test 4,Test 5,Parental consent,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder
str,i64,str,str,str,str,f64,str,str,str,i64,i64,str,str,str,str,str,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,f64,str,bool,bool,bool,bool,bool,str
"""PID0x4175""",6,"""No""","""Yes""","""No""","""No""",4.981655,"""Charles""",null,"""Kore""",38,61,"""St. Elizabeth's Hospital""","""30 WARREN ST ALLSTON/BRIGHTON,…","""Alive""","""Tachypnea""","""Normal""",0,-99,0,1,0,"""-99""","""Low""","""Male""","""Yes""","""Not applicable""","""Institute""","""Yes""","""No""","""Yes""","""-""","""No""","""-99""",2,"""Multiple""",-99.0,"""slightly abnormal""",true,true,true,true,true,"""Multifactorial genetic inherit…"
"""PID0x21f5""",10,"""Yes""","""No""",null,"""Yes""",5.11889,"""Catherine""",null,"""Homero""",33,53,"""-99""","""249 RIVER ST MATTAPAN, MA 0212…","""Alive""",null,"""-99""",0,0,-99,1,-99,"""Yes""","""Low""","""Male""","""Yes""","""Not applicable""","""-99""","""Yes""","""No""","""-99""","""-99""","""No""","""Yes""",-99,"""Multiple""",8.179584,"""normal""",false,false,false,true,false,"""Multifactorial genetic inherit…"
"""PID0x49b8""",5,"""No""",null,"""No""","""No""",4.876204,"""James""",null,"""Danield""",48,60,null,"""1400 VFW Parkway West Roxbury,…","""Deceased""",null,"""Normal""",0,0,0,1,0,"""-99""","""Low""","""Ambiguous""","""Not available""","""-99""","""Institute""","""No""","""Yes""","""Yes""","""Yes""","""Yes""","""No""",0,"""Singular""",-99.0,"""slightly abnormal""",false,false,true,true,false,"""Multifactorial genetic inherit…"
"""PID0x2d97""",13,"""No""","""Yes""","""Yes""","""No""",4.687767,"""Brian""",null,"""Orville""",25,55,"""Boston Specialty & Rehabilitat…","""51 BLOSSOM ST CENTRAL, MA 0211…","""Alive""","""-99""","""-99""",0,0,0,1,0,"""-99""","""Low""","""Ambiguous""","""No""","""Not applicable""","""-99""","""Yes""","""Yes""","""-""","""-99""","""-99""","""Yes""",-99,"""Singular""",6.884071,"""normal""",true,false,true,false,true,"""Single-gene inheritance diseas…"
"""PID0x58da""",5,"""No""",null,null,"""Yes""",5.152362,"""Gary""",null,"""Issiah""",41,38,"""Not applicable""","""-""","""Deceased""","""Tachypnea""",null,0,0,0,1,0,"""Yes""","""Low""","""Ambiguous""","""No""","""None""","""Home""","""Yes""","""Yes""","""Yes""","""Not applicable""","""No""","""No""",-99,"""Multiple""",6.195178,"""normal""",true,true,true,true,false,"""Mitochondrial genetic inherita…"


In [5]:
# Conversiones de tipos
dataset1 = dataset1.with_columns(
    cs.contains("Age", "age").cast(pl.Int64),
    cs.starts_with("Test").cast(pl.Int64),
    cs.starts_with("Symptom").cast(pl.Boolean),
    cs.by_name("No. of previous abortion").cast(pl.Int64),
)

### 1. Clasificación de desórdenes genéticos

Usando `dataset_2.tsv`, crea una nueva columna llamada "Disorder Subclass" y asigna valores según el tipo de desorden genético de cada registro:

- Cuando el desorden genético sea "Multifactorial genetic inheritance disorders", asigna "Diabetes" en la columna disorder subclass.
- Cuando el desorden genético sea "Mitochondrial genetic inheritance disorders", asigna "Mitochondrial myopathy" en la columna disorder subclass.
- Cuando el desorden genético sea "Single-gene inheritance diseases", asigna "Cystic fibrosis" en la columna disorder subclass.


In [6]:
dataset2 = dataset2.with_columns(
    pl.when(pl.col("Genetic Disorder") == "Multifactorial genetic inheritance disorders")
    .then(pl.lit("Diabetes"))
    .when(pl.col("Genetic Disorder") == "Mitochondrial genetic inheritance disorders")
    .then(pl.lit("Mitochondrial myopathy"))
    .when(pl.col("Genetic Disorder") == "Single-gene inheritance diseases")
    .then(pl.lit("Cystic fibrosis"))
    .alias("Disorder Subclass")
)

### 2. Integración y transformación de datos

Une verticalmente `dataset_1` y `dataset_2` en un único DataFrame (concatenación) y utiliza este DataFrame resultante para realizar las siguientes operaciones:

- Selecciona solo aquellos pacientes que cuenten con consentimiento de los padres y cuyo lugar de nacimiento haya sido un instituto.

- Añade una nueva columna (`age_group`) que clasifique a los pacientes por grupo etario. Para ello, utiliza la siguiente clasificación:

  - "Newborn": 0 años
  - "Early childhood": 1 - 7 años
  - "Late childhood": 8 años o más

- Integra la información del vecindario de cada paciente a partir del archivo `institute_neighborhoods.csv`. Excluye aquellos registros que no tengan información de vecindario. Deja los valores de los vecindarios en mayúsculas.

- Cambia el tipo de dato de todas las columnas que puedan representarse con un tipo más apropiado:

  - Aquellas columnas que puedan representarse con un tipo lógico (True / False) y no lo estén, cámbialas a tipo booleano.
  - Aquellas columnas que almacenan valores numéricos enteros y se encuentren con otro tipo de dato (como float o string), transfórmalas a entero.


In [ ]:
dataset = (
    pl.concat([dataset1, dataset2], how="vertical")
    .filter(
        pl.col("Parental consent") == "Yes",
        pl.col("Place of birth") == "Institute",
    )
    .with_columns(
        age_group=pl.when(pl.col("Patient Age") == 0)
        .then(pl.lit("Newborn"))
        .when(pl.col("Patient Age") <= 7)
        .then(pl.lit("Early childhood"))
        .otherwise(pl.lit("Late childhood"))
    )
    .join(
        neighborhoods,
        left_on="Location of Institute",
        right_on="institute",
        how="left",
    )
    .drop_nulls(["neighborhood"])
    .with_columns(pl.col("neighborhood").str.to_uppercase())
)

In [8]:
# ¿Cuáles columnas string tienen sólo Yes, No y -99 en sus valores?
for col in dataset.select(cs.string()).columns:
    unique = dataset.select(pl.col(col).unique()).to_series().drop_nulls().to_list()
    if set(unique).issubset({"Yes", "No", "-99"}):
        print(f"'{col}' has {unique}")

'Genes in mother's side' has ['Yes', 'No']
'Inherited from father' has ['Yes', 'No']
'Maternal gene' has ['No', 'Yes']
'Paternal gene' has ['Yes', 'No']
'Parental consent' has ['Yes']
'Folic acid details (peri-conceptional)' has ['No', 'Yes', '-99']
'H/O serious maternal illness' has ['-99', 'No', 'Yes']
'Assisted conception IVF/ART' has ['-99', 'No', 'Yes']
'History of anomalies in previous pregnancies' has ['-99', 'Yes', 'No']


In [9]:
# Conversiones de tipos (Yes/No/-99 a Boolean, las edades ya las corregimos antes)
dataset = dataset.with_columns(
    cs.by_name(
        "Genes in mother's side",
        "Inherited from father",
        "Maternal gene",
        "Paternal gene",
        "Parental consent",
        "Folic acid details (peri-conceptional)",
        "H/O serious maternal illness",
        "Assisted conception IVF/ART",
        "History of anomalies in previous pregnancies",
    )
    .replace({"Yes": 1, "No": 0, "-99": None})
    # Hay que castear a Int64 primero para evitar errores de casteo directo de string a Boolean
    .cast(pl.Int64)
    .cast(pl.Boolean),
)

### 3. Tabla resumen de pacientes

Con base en el DataFrame creado en el punto 2, crea una tabla resumen que presente la cantidad de pacientes agrupados por desorden genético, categorización de edad, vecindario y género. Descarta los registros que no tengan información del desorden genético o del género. El DataFrame debe tener la siguiente estructura:

| genetic_disorder                             | age_group       | neighborhood | male | female | ambiguous | total |
| :------------------------------------------- | --------------- | ------------ | ---- | ------ | --------- | ----- |
| Multifactorial genetic inheritance disorders | Newborn         | ROXBURY      | 35   | 27     | 20        | 82    |
| Multifactorial genetic inheritance disorders | Early childhood | SOUTH END    | 3    | 6      | 8         | 17    |
| Mitochondrial genetic inheritance disorders  | Newborn         | ROSLINDALE   | 29   | 28     | 26        | 83    |
| ...                                          |                 |              |      |        |           |       |

La columna `total` debe contener la suma de las columnas `male`, `female` y `ambiguous`. Rellena con 0 los valores faltantes en las columnas de conteo. Ordena la tabla descendentemente según la columna `total`.


In [10]:
patient_summary = (
    dataset.group_by("Genetic Disorder", "age_group", "neighborhood", "Gender")
    .len("n")
    .drop_nulls(["Genetic Disorder", "Gender"])
    .filter(pl.col("Gender") != "-99")
    .pivot(on="Gender", index=["Genetic Disorder", "age_group", "neighborhood"])
    .select(
        genetic_disorder="Genetic Disorder",
        age_group="age_group",
        neighborhood="neighborhood",
        male="Male",
        female="Female",
        ambiguous="Ambiguous",
    )
    .with_columns(total=pl.sum_horizontal(cs.numeric()))
    .sort("total", descending=True)
)
patient_summary

genetic_disorder,age_group,neighborhood,male,female,ambiguous,total
str,str,str,u32,u32,u32,u32
"""Mitochondrial genetic inherita…","""Late childhood""","""CENTRAL""",145,151,152,448
"""Mitochondrial genetic inherita…","""Late childhood""","""FENWAY/KENMORE""",147,135,128,410
"""Mitochondrial genetic inherita…","""Early childhood""","""FENWAY/KENMORE""",139,146,123,408
"""Mitochondrial genetic inherita…","""Early childhood""","""CENTRAL""",134,110,142,386
"""Single-gene inheritance diseas…","""Late childhood""","""FENWAY/KENMORE""",118,105,116,339
…,…,…,…,…,…,…
"""Multifactorial genetic inherit…","""Newborn""","""ROXBURY""",4,1,2,7
"""Multifactorial genetic inherit…","""Newborn""","""ROSLINDALE""",null,5,1,6
"""Multifactorial genetic inherit…","""Newborn""","""DORCHESTER""",1,2,3,6


### 4. Análisis de factores de riesgo

Genera un DataFrame que contenga tres columnas: `birth_defects`, `history_of` y `count`. La columna `history_of` debe contener la información:

- H/O serious maternal illness
- H/O radiation exposure (x-ray)
- H/O substance abuse

Cuando no haya defectos de nacimiento, la columna `birth_defects` debe contener el valor "No". La columna `count` debe contener la cantidad de registros en los que la columna correspondiente de historial (`history_of`) tenga un valor positivo. Descarta los registros que contengan valores inválidos o faltantes como "-99", "-", espacios vacíos, etc.

Añade la columna `percentage_by_ho` con el porcentaje que representa cada fila con respecto al total de registros por H/O.


In [11]:
dataset.select("Birth defects", cs.starts_with("H/O")).drop_nulls()

Birth defects,H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse
str,bool,str,str
"""Singular""",true,"""-""","""Not applicable"""
"""Multiple""",false,"""No""","""No"""
"""Multiple""",true,"""No""","""No"""
"""Multiple""",false,"""No""","""No"""
"""Singular""",false,"""Yes""","""No"""
…,…,…,…
"""Multiple""",false,"""Not applicable""","""Yes"""
"""Multiple""",false,"""-""","""-99"""
"""Singular""",true,"""Not applicable""","""Yes"""


In [12]:
dataset.group_by("Birth defects").len()

Birth defects,len
str,u32
"""Singular""",5251
"""Multiple""",5248
"""-99""",610
null,837


In [ ]:
ho_summary = (
    dataset.select(cs.starts_with("H/O"), birth_defects="Birth defects")
    .with_columns(
        # Asumimos que nos -99 son errores de registro, no nulos, así que no los reemplazamos
        birth_defects=pl.col("birth_defects").fill_null("No")
    )
    .unpivot(index="birth_defects", variable_name="history_of")
    .drop_nulls()
    .filter(
        pl.col("value").is_in(["Yes", "true"]),
        pl.col("birth_defects") != "-99",
    )
    .group_by("birth_defects", "history_of")
    .len("count")
    .with_columns(percentage_by_ho=pl.col("count") / pl.col("count").sum().over("history_of") * 100)
    .sort("history_of", "birth_defects")
)
ho_summary

birth_defects,history_of,count,percentage_by_ho
str,str,u32,f64
"""Multiple""","""H/O radiation exposure (x-ray)""",1142,45.174051
"""No""","""H/O radiation exposure (x-ray)""",187,7.397152
"""Singular""","""H/O radiation exposure (x-ray)""",1199,47.428797
"""Multiple""","""H/O serious maternal illness""",2317,46.563505
"""No""","""H/O serious maternal illness""",361,7.254823
"""Singular""","""H/O serious maternal illness""",2298,46.181672
"""Multiple""","""H/O substance abuse""",1136,46.272912
"""No""","""H/O substance abuse""",172,7.00611
"""Singular""","""H/O substance abuse""",1147,46.720978


### 5. Procesamiento de información de visitas

En el archivo `last_visit_info.csv` encontrarás el detalle de las últimas visitas de los pacientes. Este archivo contiene dos columnas:

- `Patient Id`: Identificador del paciente, que coincide con la columna `Patient Id` del DataFrame creado en el punto 2.
- `LastVisitInfo`: Información de la última visita en formato JSON. Contiene la fecha de la última visita, nombre del doctor, razón, entre otros.

Extrae la fecha de la última visita del JSON y guárdala en una nueva columna llamada `last_visit_date`, y haz lo mismo con `next_appointment`. Posteriormente, integra esta información al DataFrame creado en el punto 2 usando el identificador del paciente.

Finalmente, responde: ¿Cuántos pacientes del grupo etario "Early childhood" tienen una cita programada para noviembre o diciembre de 2025, y además la diferencia en días entre su última visita y la cita programada es menor a 150 días?


In [14]:
dataset.head()

Patient Id,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Patient First Name,Family Name,Father's name,Mother's age,Father's age,Institute Name,Location of Institute,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Test 1,Test 2,Test 3,Test 4,Test 5,Parental consent,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass,age_group,neighborhood
str,i64,bool,bool,bool,bool,f64,str,str,str,i64,i64,str,str,str,str,str,i64,i64,i64,i64,i64,bool,str,str,str,str,str,bool,bool,str,str,bool,bool,i64,str,f64,str,bool,bool,bool,bool,bool,str,str,str,str
"""PID0x6418""",2,true,false,true,false,4.760603,"""Richard""",null,"""Larre""",null,null,"""Boston Specialty & Rehabilitat…","""55 FRUIT ST CENTRAL, MA 02114 …","""Alive""","""Normal (30-60)""","""Normal""",0,null,null,1,0,true,"""High""",null,null,"""Not applicable""","""Institute""",false,null,"""No""","""No""",false,true,null,null,9.857562,null,true,true,true,true,true,"""Mitochondrial genetic inherita…","""Leber's hereditary optic neuro…","""Early childhood""","""CENTRAL"""
"""PID0x4ac8""",12,true,false,true,false,4.70528,"""Jeffery""","""Hoelscher""","""Aayaan""",21,null,null,"""55 FRUIT ST CENTRAL, MA 02114 …","""Deceased""","""Tachypnea""","""Normal""",0,0,0,1,0,true,"""High""","""Male""","""Not available""","""No""","""Institute""",false,true,"""-""","""Not applicable""",null,true,1,"""Singular""",7.919321,"""inconclusive""",false,false,true,false,false,"""Mitochondrial genetic inherita…","""Leigh syndrome""","""Late childhood""","""CENTRAL"""
"""PID0x44fe""",14,true,false,true,false,5.103188,"""Richard""",null,"""Coleston""",null,null,"""Massachusetts General Hospital""","""55 FRUIT ST CENTRAL, MA 02114 …","""Deceased""",null,"""Normal""",0,0,0,1,0,true,"""Low""","""Female""","""Not available""","""None""","""Institute""",false,false,"""No""","""No""",null,false,0,"""Multiple""",10.27223,"""normal""",true,false,false,true,false,"""Single-gene inheritance diseas…","""Cystic fibrosis""","""Late childhood""","""CENTRAL"""
"""PID0x8ce3""",11,false,false,true,false,5.209058,"""Willie""","""Camacho""","""Tr""",45,44,"""Lemuel Shattuck Hospital""","""125 NASHUA ST CENTRAL, MA 0211…","""Alive""","""Tachypnea""","""Tachycardia""",0,0,0,1,0,true,"""Low""","""Male""","""Yes""","""Not applicable""","""Institute""",true,true,"""No""","""No""",false,true,0,"""Multiple""",6.669552,"""slightly abnormal""",true,true,true,false,true,"""Mitochondrial genetic inherita…","""Leigh syndrome""","""Late childhood""","""CENTRAL"""
"""PID0x8660""",4,false,true,true,true,4.752272,"""John""","""Sandoval""","""Gregori""",44,42,"""Shriners Burns Institute""","""1200 Centre St Roslindale, MA …","""Alive""","""Tachypnea""","""Tachycardia""",0,0,0,1,0,true,"""Low""","""Male""","""No""","""Not applicable""","""Institute""",true,false,"""No""","""No""",true,true,1,"""Multiple""",6.397702,"""abnormal""",false,false,true,true,true,"""Multifactorial genetic inherit…","""Diabetes""","""Early childhood""","""ROSLINDALE"""


In [15]:
# Función para extraer campos de un string en formato JSON
def extract_field(json_string, field_name):
    data = json.loads(json_string)
    return data.get(field_name, None)

In [16]:
last_visit_info = last_visit_info.with_columns(
    last_visit_date=pl.col("LastVisitInfo")
    .map_elements(
        lambda x: extract_field(x, "last_visit_date"),
        return_dtype=pl.String,
    )
    .cast(pl.Date),
    next_appointment=pl.col("LastVisitInfo")
    .map_elements(
        lambda x: extract_field(x, "next_appointment"),
        return_dtype=pl.String,
    )
    .cast(pl.Date),
)

In [17]:
dataset_with_visit_info = dataset.join(
    last_visit_info.drop("LastVisitInfo"),
    on="Patient Id",
    how="left",
)

In [ ]:
patients_with_appointments_soon = dataset_with_visit_info.filter(
    pl.col("age_group") == "Early childhood",
    pl.col("next_appointment").is_between(date(2025, 11, 1), date(2025, 12, 31)),
    (pl.col("next_appointment") - pl.col("last_visit_date")).dt.total_days() < 150,
)

# NOTA: Otra opción para el filtro de fecha habría sido usar .dt.month() y .dt.year():
# pl.col("next_appointment").dt.year() == 2025,
# pl.col("next_appointment").dt.month().is_in([11, 12]),

print(f"{patients_with_appointments_soon.height} pacientes tienen citas programadas para finales de 2025.")

71 pacientes tienen citas programadas para finales de 2025.
